In [1]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

Using TensorFlow backend.


ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [ ]:
import numpy as np
import time
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
# from sklearn.cross_validation import train_test_split
import os

data_path = r'/home/garycc227/code/transferL/training pics'
data = os.listdir(data_path)

img_data_list = []

for dataset in data:
    img_list = os.listdir(data_path+'/'+dataset)
    print('loaded images' + '{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path+'/'+dataset+'/'+img
        img = image.load_img(img_path, target_size=(224,224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
#         print('Input image shape', x.shape)
        img_data_list.append(x)

img_data = np.array(img_data_list)
print(img_data.shape)
img_data = np.rollaxis(img_data,1,0)
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)


num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')

labels[0:202] = 0
labels[202:404]= 1
labels[404:606]=2

names=['cats','dogs','horses']

Y = np_utils.to_categorical(labels, num_classes)

x,y = shuffle(img_data,Y, random_state=2)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state = 2)

image_input = Input(shape = (224,224, 3))
model = VGG16(input_tensor = image_input, weights = 'imagenet')
model.summary()
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input,out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False
    
print(custom_vgg_model.layers[3].trainable)

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t = time.time()
hist = custom_vgg_model.fit(X_train,y_train, batch_size = 32 , epochs=1, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' %(t-time.time()))
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size = 10, verbose=1)

print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy * 100))


In [16]:
image_input = Input(shape=(224,224,3))

model =  VGG16(input_tensor=image_input, include_top=True, weights='imagenet')
model.summary()

last_layer= model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)
custom_vgg_model2.summary()

for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False
    
custom_vgg_model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

t = time.time()
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=1, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t-time.time()))
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size = 10, verbose = 1)

print( 'loss = {:.4f}, accuracy: {:.4f}%'.format(loss,accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________